In [3]:
from datetime import date
import deeplabcut
import glob
from subprocess import Popen, PIPE
import os
import yaml

from paramiko import SSHClient
import paramiko
from scp import SCPClient
import getpass

In [4]:
# select folder that contains videos and videos. Also select body_parts to label
# Moves to that folder so that DLC places videos and other data in that folder

vid_main_folder = r'D:\Lab\Data\DLC_videos\Han_reaching3D_202012'
videos = glob.glob(vid_main_folder + r'\*.avi')

body_part_list = ['shoulder','arm1','arm2','elbow1','elbow2',
                  'wrist1','wrist2','hand1','hand2','hand3','pointX','pointY','pointZ']

print(videos)

['D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00006.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00007.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00008.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00009.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00010.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00011.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00012.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201203_00013.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201204_00001.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201204_00002.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201204_00003.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201204_00004.avi', 'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012\\Han_20201204_00005.avi', 'D:\\Lab\\D

In [12]:
### Open this part only when you want to start to label a group of new videos and want to use this script

project = 'Han_202012'
experimenter = 'Joe'
date_used = str(date.today())
#date_used = '2020-12-04'
#path_config_file = deeplabcut.create_new_project(project, experimenter, videos,working_directory=vid_main_folder,copy_videos=True)

path_config_file = r'D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\config.yaml'

In [4]:
# adjust path_config_file to include desired body_parts and skeleton, numframes2pick, TrainingFraction, iteration
with open(path_config_file) as file:
    data = yaml.load(file, Loader = yaml.FullLoader)

data['bodyparts'] = body_part_list
data['TrainingFraction'] = [0.95] # default = 0.95
data['numframes2pick'] = 8 # default = 20


with open(path_config_file,'w') as file:
    yaml.dump(data,file)

In [5]:
"""
k-means extract 100 frames (numbers can be changed in config). 
Uses videos in path_config_file, use add_new_video to add videos to the config file
"""
deeplabcut.extract_frames(path_config_file,mode='automatic',userfeedback=False)

11it [00:00, 107.94it/s]

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 987.2  seconds.
Extracting and downsampling... 24680  frames from the video.


24680it [02:48, 146.83it/s]


Kmeans clustering ... (this might take a while)


14it [00:00, 138.49it/s]

...

Kmeans clustering ... (this might take a while)


12it [00:00, 115.36it/s]

Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 1179.68  seconds.
Extracting and downsampling... 29492  frames from the video.


29492it [03:40, 133.49it/s]


Kmeans clustering ... (this might take a while)
Frames were successfully extracted.

You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


In [7]:
"""
label frames
"""
#import matplotlib
#matplotlib.use('Agg')

deeplabcut.label_frames(path_config_file)

RuntimeError: wrapped C/C++ object of type DirDialog has been deleted

In [8]:
deeplabcut.check_labels(path_config_file)

Creating images with labels by Joe.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00006_labeled.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00007_labeled.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00008_labeled.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00009_labeled.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00010_labeled.
They are stored in the following folder: D:\Lab\Data\DLC_videos\Han_reaching3D_202012\Han_202012-Joe-2020-12-04\labeled-data\Han_20201203_00011_labeled.
They are stored in the following folder: D:\La

In [ ]:
# move files onto Shrek <- this can take awhile :(

folder_to_move = vid_main_folder + '\\' + project + '-' + experimenter + '-' + date_used

ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
#pkey = paramiko.RSAKey.from_private_key_file(pkey_filename)
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(hostname = "165.124.111.57", username= "jts3256", key_filename=pkey_filename)
scp  = SCPClient(ssh.get_transport())

scp.put(files = folder_to_move,recursive = True, 
        remote_path ="/home/jts3256/DLC_videos/")

In [10]:
vid_main_folder

'D:\\Lab\\Data\\DLC_videos\\Han_reaching3D_202012'